In [1]:
# %load PCA.py
%matplotlib inline
import pandas as pd
import numpy as np
import re
import os
import PyPDF2
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


In [3]:
stemmer=SnowballStemmer("english")
stop = set(stopwords.words('english'))
def pdf_to_text(pdf):

    pdfReader = PyPDF2.PdfFileReader(pdf)
    numPage=pdfReader.numPages
    pdftext=" "
    for page in range(numPage):
        pageObj =pdfReader.getPage(page)
        text= pageObj.extractText()
        pagetext="".join(text)
        pdftext=" ".join([pagetext, pdftext])
    alltext = re.sub('[\n]', '', pdftext)
    all_words=alltext.split(" ")
    
    affterstemmer=[]
    for word in all_words:
        affterstemmer.append(stemmer.stem(word))
    
    afterstop=[]
    for word in affterstemmer:
        if word not in stop:
            afterstop.append(word)
    return " ".join(afterstop)
THIS_FOLDER = os.path.dirname(os.path.abspath("PCA.ipynb"))



## Geting ready the training data
data=pd.read_csv('resumes.csv')
X_train=[]
for i in data['path']:
    my_file = os.path.join(THIS_FOLDER, i)
    pdf = open(my_file, 'rb')
    X_train.append(pdf_to_text(pdf))




UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe4 in position 24: invalid continuation byte

In [ ]:
def get_skills(resumes):
    skill=['python', 'c++', 'java', 'javascript', 'php', 'perl', 'go', 'ruby', 'c#']
    skill_matrix = [[0 for _ in range(len(skill))] for _ in range(len(resumes))]
    for i in range(len(resumes)):
        for j in range(len(skill)):
            if skill[j] in resumes[i]:
                skill_matrix[i][j]=1
    return skill, skill_matrix

In [21]:
skills , matrix= get_skills(X_train)

In [23]:
matrix[:10]

[[1, 1, 1, 1, 0, 0, 1, 1, 0],
 [0, 0, 0, 0, 0, 1, 1, 0, 0],
 [1, 0, 1, 1, 0, 1, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 1, 1, 0, 0, 0, 1, 0, 1],
 [0, 0, 1, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 1, 1, 0, 0, 1, 1, 0],
 [1, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [24]:
google = 'c++, java, python, javascript, go'
amazon = 'java, javascript, ruby, python, perl'
microsoft = 'c++, c#, java, python '
facebook = 'php, c++, python, java, perl'
c= [google,amazon,microsoft,facebook]
skills, companies = get_skills(c)

In [25]:
companies

[[1, 1, 1, 1, 0, 0, 1, 0, 0],
 [1, 0, 1, 1, 0, 1, 0, 1, 0],
 [1, 1, 1, 0, 0, 0, 0, 0, 1],
 [1, 1, 1, 0, 1, 1, 0, 0, 0]]

In [41]:

count_vect = CountVectorizer(lowercase = False, max_df= 0.4)
tfidf_transformer = TfidfTransformer(smooth_idf=False)
X_train_counts = count_vect.fit_transform(X_train)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)



In [42]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X_train_tfidf, data['succes'], test_size = .2, random_state = 32)

In [43]:

from sklearn.svm import SVC
cvf_rbf= SVC(kernel = 'rbf', C= 1e3).fit(x_train,y_train)
cvf_linear= SVC(kernel = 'linear', C= 1e3).fit(x_train,y_train)
cvf_poly= SVC(kernel = 'poly', C= 1e3).fit(x_train,y_train)
cvf_sigmoid= SVC(kernel = 'sigmoid', C= 1e3).fit(x_train,y_train)

pred_rbf = cvf_rbf.predict(x_test)
pred_linear= cvf_linear.predict(x_test)
pred_poly= cvf_poly.predict(x_test)
pred_sigmoid= cvf_sigmoid.predict(x_test)




In [44]:
from sklearn.metrics import accuracy_score


print ('rbf:', accuracy_score(y_test, pred_rbf))
print ('linear:', accuracy_score(y_test, pred_linear))
print ('poly:', accuracy_score(y_test, pred_poly))
print ('sigmoid:', accuracy_score(y_test, pred_sigmoid))

rbf: 0.578125
linear: 0.859375
poly: 0.578125
sigmoid: 0.578125


In [37]:
clf.predict_proba(x_test)

array([[ 0.18838081,  0.81161919],
       [ 0.18838081,  0.81161919],
       [ 0.09982625,  0.90017375],
       [ 0.57386572,  0.42613428],
       [ 0.67978848,  0.32021152],
       [ 0.41380135,  0.58619865],
       [ 0.83802891,  0.16197109],
       [ 0.88013892,  0.11986108],
       [ 0.18838081,  0.81161919],
       [ 0.80170751,  0.19829249],
       [ 0.81551782,  0.18448218],
       [ 0.60041616,  0.39958384],
       [ 0.63296361,  0.36703639],
       [ 0.37953895,  0.62046105],
       [ 0.7577134 ,  0.2422866 ],
       [ 0.86988162,  0.13011838],
       [ 0.31102603,  0.68897397],
       [ 0.44013699,  0.55986301],
       [ 0.82444153,  0.17555847],
       [ 0.46020342,  0.53979658],
       [ 0.86969384,  0.13030616],
       [ 0.59455445,  0.40544555],
       [ 0.05212613,  0.94787387],
       [ 0.6101641 ,  0.3898359 ],
       [ 0.07123211,  0.92876789],
       [ 0.18838081,  0.81161919],
       [ 0.15473584,  0.84526416],
       [ 0.18838081,  0.81161919],
       [ 0.15082619,

In [12]:

s= "this is a aste'sdasd' adasd! . . asdas .. a@>< > A.< ,./ . ??? !! sadasdasd " 

re.sub('[^A-Za-z0-9]+', '', s)

'thisisaastesdasdadasdasdasaAsadasdasd'

In [23]:
stemmer=SnowballStemmer("english")
stop = set(stopwords.words('english'))
def pdf_to_text(pdf):

    pdfReader = PyPDF2.PdfFileReader(pdf)
    numPage=pdfReader.numPages
    pdftext=" "
    for page in range(numPage):
        pageObj =pdfReader.getPage(page)
        text= pageObj.extractText()
        pagetext="".join(text)
        pdftext=" ".join([pagetext, pdftext])
    alltext = re.sub('[\n]', '', pdftext)
    all_words=alltext.split(" ")
    for i in range(len(all_words)):
        all_words[i]= re.sub('[^A-Za-z]+', '', all_words[i])
    affterstemmer=[]
    for word in all_words:
        affterstemmer.append(stemmer.stem(word))
    
    afterstop=[]
    for word in affterstemmer:
        if word not in stop:
            afterstop.append(word)
    return " ".join(afterstop)
THIS_FOLDER = os.path.dirname(os.path.abspath("PCA.ipynb"))



In [24]:

## Geting ready the training data
data=pd.read_csv('resumes.csv')
X_train=[]
for i in data['path'][:5]:
    my_file = os.path.join(THIS_FOLDER, i)
    pdf = open(my_file, 'rb')
    X_train.append(pdf_to_text(pdf))






In [25]:
X_train

[' creat demonstr earthquak simul disast prepar  execut san mateocounti creat custom physic vizard virtual realiti softwar toolkit creat custom graphic blender comput graphic softwar construct virtual model citi wrote algorithm move virtual limb base hand gestur  particip studi stanforddepart ofpsycholog lead new research field inbodi cognitioneducationmast virtual realiti comput visionstanford univers  stanford caseptemb  presentstanford  santiago de chile regin metropolitanaseptemb  decemb london school econom  londonjuli  august skillsedit skill technic skill unreal engin uniti python java c c docker vizard blender androidgit matlab rubi assembl web applic develop skill meteor rubi rail html cssjavascript sql coffeescript blaze semant ui less  year javascript  yearslinkshttpswwwlinkedincominabhigarg abhi gargsoftwar engineerstanford caemail inde indeedcomrabhigargabcdafwork experiencesoftwar engineerqualia lab  san francisco cajun  septemb  creat imag process softwar autom part newc